In [1]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim import Adam
import torch.nn.functional as F

import csv
from skimage import io

from PIL import Image
import pandas as pd

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import matplotlib.pyplot as plt
import time
import os
import copy

import import_ipynb
import ResNetCaps_IBN
import Pets_Loader
import Animals_Loader
import Marvel_Loader


verbose = False
USE_CUDA = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

CIFAR10_USE = False
MARVEL_USE =  True
PETS_USE = False
Animals_USE = False



def lr_decrease(optimizer, lr_clip):  
    for param_group in optimizer.param_groups:
        init_lr = param_group['lr'] 
        param_group['lr'] = init_lr*lr_clip
        
def isnan(x):
    return x != x

def resume_model(name_file, model, optimizer): 
    if os.path.isfile(name_file):
        print("=> loading checkpoint '{}'".format(name_file))
        checkpoint = torch.load(name_file)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
              .format(name_file, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(name_file))

    return start_epoch,model,optimizer

dataset_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),        
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])


batch_size = 130


if CIFAR10_USE: 
    NUM_CLASSES = 10
    print("CIFAR10")
    name_dataset = "CIFAR10"
    image_datasets = {'train': datasets.CIFAR10('../data', train=True, download=True, transform=dataset_transform),'val': datasets.CIFAR10('../data', train=False, download=True, transform=dataset_transform)}
    print("Initializing Datasets and Dataloaders...")
    dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True) , 'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True) }
    print("Initializing Datasets and Dataloaders...")

if MARVEL_USE: 
    
    NUM_CLASSES = 26
    print("MARVEL")
    name_dataset = "MARVEL"
    dat_file = "/home/rita/JupyterProjects/EYE-SEA/DataSets/marveldataset2016-master/FINAL.dat"
    #dat_file = "/media/Data/rita/EYE-SEA/Verification/Datasets/marveldataset2016-master/FINAL.dat"
    image_datasets = {'train': Marvel_Loader.MARVEL_dataset(dat_file,train = True,transform=dataset_transform),'val': Marvel_Loader.MARVEL_dataset(dat_file,train = False,transform=dataset_transform)}
    print("Initializing Datasets and Dataloaders...")
    dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True) , 'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True) }
    print("Initializing Datasets and Dataloaders...")

if  PETS_USE:
    NUM_CLASSES = 37
    print("PETS")
    name_dataset = "PETS"    
    dat_file = "/home/rita/JupyterProjects/EYE-SEA/DataSets/Pets/Pet_Datasets"
    #dat_file = "/media/Data/rita/EYE-SEA/Verification/Datasets/Pets/Pet_Datasets"
    image_datasets = {'train': Pets_Loader.PETS_dataset(dat_file,train = True,transform=dataset_transform),'val': Pets_Loader.PETS_dataset(dat_file,train = False,transform=dataset_transform)}
    print("Initializing Datasets and Dataloaders...")
    dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True) , 'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True) }
    print("Initializing Datasets and Dataloaders...")    
    
if  Animals_USE:
    NUM_CLASSES = 50
    print("Animals")
    name_dataset = "Animals"     
    dat_file = "/home/rita/JupyterProjects/EYE-SEA/DataSets/Animals_with_Attributes2/JPEGImages"
    #dat_file = "/media/Data/rita/EYE-SEA/Verification/Datasets/Animals_with_Attributes2/JPEGImages"
    image_datasets = {'train': Animals_Loader.Animals_dataset(dat_file,train = True,transform=dataset_transform),'val': Animals_Loader.Animals_dataset(dat_file,train = False,transform=dataset_transform)}
    print("Initializing Datasets and Dataloaders...")
    dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True) , 'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True) }
    print("Initializing Datasets and Dataloaders...")    


dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

not_ibn = False
model_path =  "/home/rita/JupyterProjects/EYE-SEA/ResNet_CAPSNET/PY/ResNetCaps/"+name_dataset+"/checkpoint_99.pth.tar"

importing Jupyter notebook from ResNetCaps_IBN.ipynb
importing Jupyter notebook from CapsNet_Layers.ipynb
importing Jupyter notebook from Pets_Loader.ipynb
importing Jupyter notebook from Animals_Loader.ipynb
importing Jupyter notebook from Marvel_Loader.ipynb
MARVEL
Initializing Datasets and Dataloaders...
Initializing Datasets and Dataloaders...


In [3]:
print("ResNetCaps_IBN")
model =  ResNetCaps_IBN.IBN_ResNetCaps(NUM_CLASSES)
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),lr = 0.001)
start_epoch,model,optimizer = resume_model(model_path, model, optimizer)
for param in model.parameters():
    param.requires_grad = False
model = model.to(device)
criterion = nn.CrossEntropyLoss()

model.eval()
test_loss = 0
test_accuracy = 0
start = time.time()

for batch_id, (inputs, labels) in enumerate(dataloaders['val']):
    if MARVEL_USE: labels = labels-1    
    labels =torch.eye(NUM_CLASSES).index_select(dim=0, index=labels)
    inputs, labels = Variable(inputs), Variable(labels)
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    masked = model.decoder(outputs, inputs)
    
    loss = model.model_loss(outputs, labels)
    test_loss += loss.data
    test_accuracy += (sum(np.argmax(masked.data.cpu().numpy(), 1) == np.argmax(labels.data.cpu().numpy(), 1)) / float(batch_size))
    
    if batch_id % 100 == 0:
            print("test accuracy:", sum(np.argmax(masked.data.cpu().numpy(), 1) == 
                                   np.argmax(labels.data.cpu().numpy(), 1)) / float(batch_size) )          
end = time.time()   
print("Validation time execution {}".format(end-start))
print("Loss value for test phase: {}".format(test_loss / len(dataloaders['val'])))
print("Accuracy value for test phase: {}".format(test_accuracy / len(dataloaders['val'])))





ResNetCaps_IBN
=> loading checkpoint '/home/rita/JupyterProjects/EYE-SEA/ResNet_CAPSNET/PY/ResNetCaps/MARVEL/checkpoint_99.pth.tar'
=> loaded checkpoint '/home/rita/JupyterProjects/EYE-SEA/ResNet_CAPSNET/PY/ResNetCaps/MARVEL/checkpoint_99.pth.tar' (epoch 100)


CapsNet_Layers.ipynb:96: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  "        u_hat = torch.matmul(W, x)\n",
ResNetCaps_IBN.ipynb:78: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  "\n",


test accuracy: 0.6076923076923076
test accuracy: 0.6
test accuracy: 0.7307692307692307
Validation time execution 149.18897795677185
Loss value for test phase: 0.4522314965724945
Accuracy value for test phase: 0.6523304281924972


In [ ]:
torch.cuda.empty_cache()